In [1]:
import pandas as pd
import regex as re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/amiralid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!unzip data/amazon_reviews.csv.zip

unzip:  cannot find or open data/amazon_reviews.csv.zip, data/amazon_reviews.csv.zip.zip or data/amazon_reviews.csv.zip.ZIP.


In [3]:
df = pd.read_csv('amazon_reviews.csv')

In [4]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [5]:
# Keep only reviewText column
df = df[['reviewText']]

In [6]:
df.head()

,reviewText
0,No issues.
1,"Purchased this for my device, it worked as adv..."
2,it works as expected. I should have sprung for...
3,This think has worked out great.Had a diff. br...
4,"Bought it with Retail Packaging, arrived legit..."


In [7]:
def process(text: str):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[\/|-|>|<|=|\(|\)|\{|\}|\[\]]+', '', text) # Eliminate slash and hyphen and other chars
    text = re.sub(r'[0-9]+(gb|GB|Gb)', '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[0-9]{1,2}\/[0-9]{1,2}\/[0-9]{2}', '', text)  # Remove dates
    
    if not text.endswith('.'):
        text += '.'  # Add a period to the end of the sentence if it doesn't have one
    text += " "
    return text

# Apply the function to each item in the column and then concatenate the results
running_string = "".join(df['reviewText'].apply(lambda x: process(str(x))))

In [8]:
sentences = sent_tokenize(running_string)

In [9]:
sentences = sentences[:9000]

In [10]:
sentences[:10]

['No issues.',
 'Purchased this for my device, it worked as advertised.',
 'You can never have too much phone memory, since I download a lot of stuff this was a no brainer for me.',
 'it works as expected.',
 'I should have sprung for the higher capacity.',
 'I think its made a bit cheesier than the earlier versions; the paint looks not as clean as before.',
 'This think has worked out great.Had a diff.',
 'bran  card and if went south after  months.This one has held up pretty well since I had my S, now on my Note.',
 "*** update I've had this for a few months and have had ZERO issue's since it was transferred from my S to my Note and into a note.",
 'This card is reliable and solid!Cheers!.']

In [11]:
sentences[100:110]

['This item is great!',
 "I can't believe how small it is.",
 "Imagine  this size back in the 's when they had whole buildings full of computers that couldn't pack this much space!",
 'Great item.',
 "I've been trying for a while to get a hold of a  Micro SD for my phone and tablet.",
 "It's pretty fast compared to the  cards I've been using.",
 'I really saw the performance boost with my phone, which I have many apps, pictures, music, and other documents on.',
 'The phone boots fully stable in about  mins as apposed to - mins.',
 'My computer also can write to these cards much faster than the  ones I was using.',
 'If you phone, tablet or any other device can support it I recommend you purchase these.']

In [12]:
tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

In [13]:
all_words = [word.strip() for sentence in tokenized_sentences for word in sentence]
vocabulary = set(all_words)

In [14]:
all_words[115:135]

['for',
 'a',
 'few',
 'months',
 'and',
 'have',
 'had',
 'ZERO',
 'issue',
 "'s",
 'since',
 'it',
 'was',
 'transferred',
 'from',
 'my',
 'S',
 'to',
 'my',
 'Note']

In [15]:
len(vocabulary)

8198

In [16]:
# Using a list comprehension, we can loop through each (idx, word) pair from enumerate
# Each pair holds the key and value we want in our word_to_idx dict
# We start the enumeration from 1, not 0, because want 0 to represent the padding token
word_to_idx = {word.strip() : idx for idx, word in enumerate(vocabulary, 1)}
# Let's also create a idx_to_word dict so we can interpet the results of the model later
idx_to_word = {idx : word.strip() for word, idx in word_to_idx.items()}
vocab_size = len(vocabulary) + 1

In [17]:
import pickle
with open('amzn_word_to_idx.pkl', 'wb') as f:  # open a text file
    pickle.dump(word_to_idx, f)
with open('amzn_idx_to_word.pkl', 'wb') as f:
    pickle.dump(idx_to_word, f)

In [18]:
input_sequences = []
for sentence in tokenized_sentences:
  # Convert the sentence to its numerical representation with the word_to_idx mapping
  numerized_sentence = [word_to_idx[word.strip()] for word in sentence]
  # Create ngrams from size 2 to the size of the sentence
  for i in range(2, len(sentence) + 1):
    ngram = numerized_sentence[:i]
    input_sequences.append(ngram)

In [19]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
# Now, let's pad the sequences so they are all the same length
max_sequence_len = max([len(seq) for seq in input_sequences])
# pad_sequences adds 0s to the beginning of each array until size(vector) = max_sequence_len
# This is why we started our enumeration from 1, not 0, because 0 represents the padding token
# We use pre padding because padding at the end would cause us to lose the location of the label
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [21]:
X = [sequence[:-1] for sequence in input_sequences]
y = [sequence[-1] for sequence in input_sequences]

In [22]:
def pretty_display(vec: list[int], idx_to_word: dict):
    print(" ".join([idx_to_word[idx] if idx else '' for idx in vec]).strip())

def pretty_display_all(vecs: list[list[int]], idx_to_word: dict):
    for vec in vecs:
        pretty_display(vec, idx_to_word)

def pretty_display_one(idx: int, idx_to_word: dict):
    print(idx_to_word[idx])

In [23]:
pretty_display(X[1003], idx_to_word)

I would not hesitate to buy


In [24]:
pretty_display(y[1003:1008], idx_to_word)

another one of these great


In [25]:
y = to_categorical(y, num_classes=vocab_size)

In [26]:
import numpy as np

In [27]:
arr_x = np.array(X)
arr_x.shape

(140881, 198)

In [28]:
# arr_y = np.array(y)
# arr_y.shape

# Building the Model

In [29]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, BatchNormalization, SpatialDropout1D, GaussianNoise
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [30]:
# Building the RNN model
model = Sequential()

# Embedding layer
model.add(Embedding(vocab_size, 50, input_length=max_sequence_len-1))  # Increased embedding dimensions
# model.add(SpatialDropout1D(0.15))
# model.add(GaussianNoise(0.1))

# RNN 1
model.add((GRU(100, dropout=0.1, recurrent_dropout=0.1)))  # Increased units, added dropout
# model.add(BatchNormalization())

# Final Layer
model.add(Dense(vocab_size, activation='softmax'))

2023-11-15 13:24:35.001968: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 13:24:35.759685: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14623 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:3b:00.0, compute capability: 7.0


In [31]:
model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 198, 50)           409950    
                                                                 
 gru (GRU)                   (None, 100)               45600     
                                                                 
 dense (Dense)               (None, 8199)              828099    
                                                                 
Total params: 1,283,649
Trainable params: 1,283,649
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Stop the model early
early_stop = EarlyStopping(monitor='loss', patience=12, verbose=1, restore_best_weights=True)

In [33]:
from tensorflow import convert_to_tensor

In [34]:
X = convert_to_tensor(arr_x)
y = convert_to_tensor(y)

In [35]:
# This will train the model; adjust epochs and batch size as necessary
history = model.fit(X, y, epochs=100, batch_size=64, verbose=1, validation_split=0.2, callbacks=[early_stop])

Epoch 1/100
1761/1761 [==============================] - 427s 241ms/step - loss: 5.8765 - accuracy: 0.0933 - val_loss: 5.2756 - val_accuracy: 0.1421
Epoch 2/100
1761/1761 [==============================] - 422s 240ms/step - loss: 4.9935 - accuracy: 0.1619 - val_loss: 4.9721 - val_accuracy: 0.1776
Epoch 3/100
1761/1761 [==============================] - 424s 241ms/step - loss: 4.6272 - accuracy: 0.1893 - val_loss: 4.8770 - val_accuracy: 0.1885
Epoch 4/100
1761/1761 [==============================] - 423s 240ms/step - loss: 4.3736 - accuracy: 0.2077 - val_loss: 4.8305 - val_accuracy: 0.1975
Epoch 5/100
1761/1761 [==============================] - 422s 240ms/step - loss: 4.1724 - accuracy: 0.2213 - val_loss: 4.8275 - val_accuracy: 0.2032
Epoch 6/100
1761/1761 [==============================] - 424s 241ms/step - loss: 4.0088 - accuracy: 0.2321 - val_loss: 4.8427 - val_accuracy: 0.2078
Epoch 7/100
1761/1761 [==============================] - 426s 242ms/step - loss: 3.8669 - accuracy: 0.2441

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1761/1761 [==============================] - 428s 243ms/step - loss: 2.6990 - accuracy: 0.3867 - val_loss: 5.3373 - val_accuracy: 0.2033
Epoch 30/100
1516/1761 [========================>.....] - ETA: 56s - loss: 2.6645 - accuracy: 0.3921

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1761/1761 [==============================] - 429s 243ms/step - loss: 2.5613 - accuracy: 0.4077 - val_loss: 5.4718 - val_accuracy: 0.2016
Epoch 38/100
1239/1761 [====================>.........] - ETA: 2:00 - loss: 2.5170 - accuracy: 0.4158

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1761/1761 [==============================] - 428s 243ms/step - loss: 2.4706 - accuracy: 0.4234 - val_loss: 5.5704 - val_accuracy: 0.2001
Epoch 45/100
1761/1761 [==============================] - 428s 243ms/step - loss: 2.4606 - accuracy: 0.4243 - val_loss: 5.5840 - val_accuracy: 0.1995
Epoch 46/100
 965/1761 [===============>..............] - ETA: 3:02 - loss: 2.4035 - accuracy: 0.4344

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1761/1761 [==============================] - 428s 243ms/step - loss: 2.4396 - accuracy: 0.4268 - val_loss: 5.6112 - val_accuracy: 0.2007
Epoch 48/100
1761/1761 [==============================] - 426s 242ms/step - loss: 2.4280 - accuracy: 0.4291 - val_loss: 5.6258 - val_accuracy: 0.2017
Epoch 49/100
1761/1761 [==============================] - 428s 243ms/step - loss: 2.4210 - accuracy: 0.4315 - val_loss: 5.6328 - val_accuracy: 0.1998
Epoch 50/100
1761/1761 [==============================] - 424s 241ms/step - loss: 2.4114 - accuracy: 0.4321 - val_loss: 5.6446 - val_accuracy: 0.1996
Epoch 51/100
1761/1761 [==============================] - 427s 242ms/step - loss: 2.4005 - accuracy: 0.4341 - val_loss: 5.6567 - val_accuracy: 0.1978
Epoch 52/100
1761/1761 [==============================] - 429s 243ms/step - loss: 2.3901 - accuracy: 0.4354 - val_loss: 5.6710 - val_accuracy: 0.1976
Epoch 53/100
1761/1761 [==============================] - 427s 243ms/step - loss: 2.3839 - accuracy: 0.4347 - val

In [36]:
model.save('amazon_reviews_GRU.h5')

In [37]:
from tensorflow.keras.models import load_model

In [38]:
model = load_model('amazon_reviews_GRU.h5')

In [39]:
file = open("amzn_idx_to_word.pkl", 'rb')
idx_to_word = pickle.load(file)
file.close()

file = open("amzn_word_to_idx.pkl", 'rb')
word_to_idx = pickle.load(file)
file.close()

In [40]:
def predict_next_n_words(model, text, n, max_sequence_len, word_to_index, index_to_word):
    """
    Predict the next n words based on the input text.

    Args:
    - model (tf.keras.Model): Trained model for prediction.
    - text (str): Input string.
    - n (int): Number of words to predict.
    - max_sequence_len (int): Maximum length of input sequences.
    - word_to_index (dict): Mapping from words to their respective indices.
    - index_to_word (dict): Mapping from indices to their respective words.

    Returns:
    - str: Predicted sequence of words.
    """

    predicted_sequence = []

    for _ in range(n):
        # Tokenize the input string
        token_list = [word_to_index[word] for word in word_tokenize(text) if word in word_to_index]

        # Pad the token sequence
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # Predict the token of the next word
        predicted_idx = np.argmax(model.predict(token_list), axis=-1)

        # Convert the token back to a word
        predicted_word = index_to_word.get(predicted_idx[0], '')

        # Append the predicted word to the sequence and to the text (for the next iteration)
        predicted_sequence.append(predicted_word)
        text += " " + predicted_word

    return ' '.join(predicted_sequence)

In [46]:
input_text = "In my opinion,"
prediction = predict_next_n_words(model, input_text, 20, max_sequence_len, word_to_idx, idx_to_word)
print(input_text + " " + prediction)

In my opinion, I can get another . , but I have to use it in my phone , and it has worked


In [42]:
print(max_sequence_len)

199
